# Get the weather indicator by country


    
# => Start by calculating mean deltas

### Metric 1.1: Exposure to temperature change

The vulnerability to climate change is a function of the physical hazard (change in temperature, precipitation, humidity etc) and the number of (vulnerable) people exposed. A first order measure of exposure is defined here as:


$$
E(x,t) = \Delta T(x,t)P(x,t)
$$

and the global average is

$$
\bar{E} = \frac{\int \Delta T(x,t)P(x,t) dx}{\int P(x,t) dx}
$$


Which is equal to

$$
\bar{E} = \frac{\int \Delta T(x,t)P(x,t) dx}{\int P(x,t) dx}
$$





In [4]:
%matplotlib inline

from pathlib import Path
from datetime import date

import numpy as np
import scipy as sp
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import rasterio
import fiona

import cartopy
from cartopy import crs

import geopandas as gpd 
from config import (DATA_SRC, POP_DATA_SRC,  ERA_MONTHLY_FILE, CLIMATOLOGY_FILE_RESAMP, 
                    SHAPEFILES_SRC, CLIMATOLOGY_FILE_MONTHLY, COUNTRY_POLY_SHP)
import weather_ecmwf
import util

In [10]:
gpd_countries = gpd.read_file(str(COUNTRY_POLY_SHP))
world_bank_yr_pop_file = POP_DATA_SRC / 'API_SP' / 'API_SP.POP.TOTL_DS2_en_csv_v2.csv'
world_bank_pop =  pd.read_csv(world_bank_yr_pop_file)
world_bank_pop = world_bank_pop.rename(columns={'Country Code': 'ISO_A3'})
gpd_countries = gpd_countries.set_index('ISO_A3').join(world_bank_pop.set_index('ISO_A3'))



In [11]:
gpd_countries

,ABBREV,ABBREV_LEN,ADM0_A3,ADM0_A3_IS,ADM0_A3_UN,ADM0_A3_US,ADM0_A3_WB,ADM0_DIF,ADMIN,BRK_A3,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,Unnamed: 61
ISO_A3,,,,,,,,,,,,,,,,,,,,,
-99,A.C.Is.,7.0,ATC,AUS,-99.0,ATC,-99.0,1.0,Ashmore and Cartier Islands,ATC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-99,None,0.0,BJN,-99,-99.0,B41,-99.0,0.0,Bajo Nuevo Bank (Petrel Is.),B41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-99,Clp. I.,7.0,CLP,FRA,-99.0,CLP,-99.0,1.0,Clipperton Island,CLP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-99,None,0.0,CNM,CYP,-99.0,CYP,-99.0,0.0,Cyprus No Mans Area,B43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-99,C.S.I.,6.0,CSI,AUS,-99.0,CSI,-99.0,1.0,Coral Sea Islands,CSI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-99,N. Cy.,6.0,CYN,CYP,-99.0,CYP,-99.0,0.0,Northern Cyprus,B20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-99,Dhek.,5.0,ESB,GBR,-99.0,ESB,-99.0,1.0,Dhekelia Sovereign Base Area,ESB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-99,Fr.,3.0,FRA,FRA,-99.0,FRA,-99.0,1.0,France,FRA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-99,Ind. Oc. Ter.,13.0,IOA,AUS,-99.0,AUS,-99.0,1.0,Indian Ocean Territories,IOA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
